### Plot data from the netCDF conversions of the original log files and compare with SSDS generated version
To run this Notebook:
- Install with instructions at https://bitbucket.org/mbari/auv-python
- `cd auv-python`
- `poetry install && poetry shell`
- `cd notebooks && jupyter notebook 3.0-mpm-quick-look-ssds.ipynb`

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f != '.DS_Store'])
display(auv_picker)

In [ ]:
# Pick auv mission on local system
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir))
                                                   if f != '.DS_Store'])
display(mission_picker)

In [ ]:
mission = mission_picker.value

# Get handle to corresponding SSDS generated netCDF
ssds_path = os.path.join('http://dods.mbari.org:80/opendap/data/ssdsdata/ssds/generated/netcdf/files',
                         'ssds.shore.mbari.org/auvctd/missionlogs/', mission.split('.')[0],
                         ''.join(mission.split('.')[:2]), mission)

# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
ncs = {}
dds = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        nc_url = os.path.join(netcdfs_dir, nc_file)
        ncs[nc_file] = xr.open_dataset(nc_url)
    except (FileNotFoundError, OSError):
        print(f'{nc_url} not found')
    try:
        dd_url = os.path.join(ssds_path, nc_file)
        dds[nc_file] = xr.open_dataset(dd_url)
    except (FileNotFoundError, OSError):
        print(f'{dd_url} not found')

In [ ]:
# Print the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)

In [ ]:
variable = var_picker.value
 
local_ds = ncs[ds_picker.value]
ssds_ds = dds[ds_picker.value]

var = getattr(local_ds, variable)
try:
    d_var = getattr(ssds_ds, variable)
except AttributeError as e:
    d_var = xr.DataArray([])

# Plot first hour of locally generated variable with the one created and served by SSDS
first_hour = local_ds.time.values[0].astype('str').split(':')[0]    
df = pd.concat([local_ds[variable].sel(time=first_hour).to_pandas().rename(f"auv-python"),
                 ssds_ds[variable].sel(time=first_hour).to_pandas().rename(f"SSDS")], axis=1)
display(df.hvplot(ylabel=f"{var.attrs['long_name']} [{var.attrs.get('units', '')}]"))